In [ ]:
from datetime import datetime, timedelta

# File paths
BOOKS_FILE = r"C:\Users\ahmed\OneDrive\Desktop\hackathon\books.txt"
MEMBERS_FILE = r"C:\Users\ahmed\OneDrive\Desktop\hackathon\members.txt"

# Book class to manage book details
class Book:
    def __init__(self, book_id, title, author, genre, available=True):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.genre = genre
        self.available = available

    def __repr__(self):
        return f"Book(ID: {self.book_id}, Title: '{self.title}', Author: '{self.author}', Genre: '{self.genre}', {'Available' if self.available else 'Unavailable'})"

    def serialize(self):
        return f"{self.book_id},{self.title},{self.author},{self.genre},{self.available}\n"

    @staticmethod
    def deserialize(book_str):
        try:
            book_id, title, author, genre, available = book_str.strip().split(',')
            return Book(book_id, title, author, genre, available.strip().lower() == "true")
        except ValueError as e:
            raise ValueError("Invalid book data format") from e

# Member class to manage member details and borrowed books
class Member:
    def __init__(self, member_id, name):
        self.member_id = member_id
        self.name = name
        self.borrowed_books = []

    def borrow_book(self, book, due_date):
        self.borrowed_books.append({"book_id": book.book_id, "due_date": due_date.strftime("%Y-%m-%d")})

    def return_book(self, book_id):
        for book in self.borrowed_books:
            if book['book_id'] == book_id:
                self.borrowed_books.remove(book)
                break

    def __repr__(self):
        return f"Member(ID: {self.member_id}, Name: '{self.name}', Borrowed Books: {self.borrowed_books})"

    def serialize(self):
        borrowed_books_str = ';'.join([f"{book['book_id']}:{book['due_date']}" for book in self.borrowed_books])
        return f"{self.member_id},{self.name},{borrowed_books_str}\n"

    @staticmethod
    def deserialize(member_str):
        try:
            member_id, name, borrowed_books_str = member_str.strip().split(',')
            member = Member(member_id, name)
            if borrowed_books_str:
                borrowed_books = borrowed_books_str.split(';')
                for book in borrowed_books:
                    book_id, due_date = book.split(':')
                    member.borrowed_books.append({"book_id": book_id, "due_date": due_date})
            return member
        except ValueError as e:
            print(f"Error parsing member entry '{member_str}': {e}")
            raise ValueError("Invalid member data format") from e

# Library class to manage the entire library system
class Library:
    late_fee_of_one_day = 1
    def __init__(self):
        self.books = {}
        self.members = {}
        self.load_data()

    def load_data(self):
        try:
            with open(BOOKS_FILE, 'r') as f:
                for line in f:
                    line = line.strip()
                    if line:  # Only process non-empty lines
                        try:
                            book = Book.deserialize(line)
                            self.books[book.book_id] = book
                        except ValueError as e:
                            print(f"Skipping invalid book entry: {line}. Error: {e}")
        except FileNotFoundError:
            print("there is an ERROR in file path please insert correct path!")
            self.books = {}

        try:
            with open(MEMBERS_FILE, 'r') as f:
                for line in f:
                    line = line.strip()
                    if line:  # Only process non-empty lines
                        try:
                            member = Member.deserialize(line)
                            self.members[member.member_id] = member
                        except ValueError as e:
                            print(f"Skipping invalid member entry: {line}. Error: {e}")
        except FileNotFoundError:
            print("there is an ERROR in file path please insert correct path!")
            self.members = {}

    def save_data(self):
        with open(BOOKS_FILE, 'w') as f:
            for book in self.books.values():
                f.write(book.serialize())
        with open(MEMBERS_FILE, 'w') as f:
            for member in self.members.values():
                f.write(member.serialize())

    def add_book(self, book_id, title, author, genre):
        if book_id in self.books:
            print(f"Book ID {book_id} already exists.")
        else:
            new_book = Book(book_id, title, author, genre)
            self.books[book_id] = new_book
            self.save_data()
            print(f"Book {title} has been added in this catalog :)")

    def update_book(self, book_id, **kwargs):
        if book_id in self.books:
            for key, value in kwargs.items():
                if value:  # Only update if the value is not empty
                    setattr(self.books[book_id], key, value)
            self.save_data()
            print(f"Book with this ID : {book_id} has been updated :)")
        else:
            print("This ID not found in this catalog :(")
        
    def remove_book(self, book_id):
        if book_id in self.books:
            del self.books[book_id]
            self.save_data()
            print(f"Book {book_id} removed.")
        else:
            print("This book with this ID not found :(")

    def register_member(self, member_id, name):
        if member_id in self.members:
            print(f"Member ID {member_id} already exists!")
        else:
            new_member = Member(member_id, name)
            self.members[member_id] = new_member
            self.save_data()
            print(f"Member {name} registered.")

    def update_member(self, member_id, **kwargs):
        if member_id in self.members:
            for key, value in kwargs.items():
                if value:  # Only update if the value is not empty
                    setattr(self.members[member_id], key, value)
            self.save_data()
            print(f"Member with this {member_id} has been updated :)")
        else:
            print("Member not found.")

    def remove_member(self, member_id):
        if member_id in self.members:
            del self.members[member_id]
            self.save_data()
            print(f"Member with this {member_id} has been removed !!")
        else:
            print("Member with this ID does not exist in this system !!")

    def borrow_book(self, book_id, member_id):
        if book_id in self.books: #and self.books[book_id].available:
            if member_id in self.members:
                self.books[book_id].available = False
                due_date = datetime.now() + timedelta(days=14)
                self.members[member_id].borrow_book(self.books[book_id], due_date)
                self.save_data()
                print(f"Book with this ID: {book_id}  has been borrowed by member with ID : {member_id}. Due on {due_date.strftime('%Y-%m-%d')}.")
            else:
                print("Member not found.")
        else:
            print("Book or member not found in the system !")

    def return_book(self, book_id, member_id):
        if member_id in self.members and book_id in self.books:
            book_info = None
            for book in self.members[member_id].borrowed_books:
                if book['book_id'] == book_id:
                    book_info = book
                    break
            
            if book_info:
                due_date = datetime.strptime(book_info['due_date'], "%Y-%m-%d")
                late_fee = self.calculate_late_fee(due_date)
                
                self.members[member_id].return_book(book_id)
                self.books[book_id].available = True
                self.save_data()
                
                if late_fee > 0:
                    print(f"Book {book_id} returned by member {member_id}. Late fee: ${late_fee}.")
                else:
                    print(f"Book {book_id} returned by member {member_id}. No late fee.")
            else:
                print(f"The book {book_id} was not borrowed by member {member_id}.")
        else:
            print("Member or book not found in this system!")

    def calculate_late_fee(self, due_date):
        current_date = datetime.now()
        if current_date > due_date:
            days_overdue = (current_date - due_date).days
            return days_overdue * self.late_fee_of_one_day
        return 0
    def search_books(self, search_term):
        found_books = []
        for book in self.books.values():
            if search_term.lower() in book.title.lower():
                found_books.append(book)
        return found_books

    def search_members(self, search_term):
        found_members = []
        for member in self.members.values():
            if search_term.lower() in member.name.lower():
                found_members.append(member)
        return found_members
    def list_borrowed_books(self):
         for member in self.members.values():
          if member.borrowed_books:
           print(f"Member {member.name} has borrowed:")
          for book in member.borrowed_books:
            print(f"- {book['book_id']} (Due: {book['due_date']})")
          else: 
           print("")
             
    def list_overdue_books(self):
        current_date = datetime.now().strftime("%Y-%m-%d")
        for member in self.members.values():
            for book in member.borrowed_books:
                if book['due_date'] < current_date:
                    print(f"Overdue book : {book['book_id']} (Due: {book['due_date']})")
          
    def most_popular_books(self):
        book_borrow_count = {}
        for member in self.members.values():
            for book in member.borrowed_books:
                book_id = book['book_id']
                book_borrow_count[book_id] = book_borrow_count.get(book_id, 0) + 1
    
        if book_borrow_count:
            most_borrowed = sorted(book_borrow_count.items(), key=lambda x: x[1], reverse=True)
            print("Most popular books (most borrowed):")
            for book_id, count in most_borrowed:
                if count > 2:  # Check if count is greater than 3
                    book = self.books.get(book_id)
                    if book:
                        print(f"- {book.title} (ID: {book_id}, Borrowed {count} times)")
        else:
            print("No books have been borrowed yet.")
        
def menu():
    library = Library()
    while True:
        print("\nLibrary Management System")
        print("--------------------------")
        print("1.  Add book")
        print("2.  Update book")
        print("3.  Remove book")
        print("4.  Register member")
        print('5.  Update member')
        print('6.  Remove member')
        print("7.  Borrow book")
        print("8.  Return book")
        print("9.  List borrowed books")
        print('10. List overdue books')
        print('11. Most popular books')
        print('12. Search for book')
        print('13. Search for member')
        print("0.  Exit")
        print("--------------------------")
        choice = input("Enter choice: ")
        if choice == '1':
            book_id = input("Book ID: ")
            title = input("Title: ")
            author = input("Author: ")
            genre = input("Genre: ")
            library.add_book(book_id, title, author, genre)
        elif choice == '2':
            book_id = input("Book ID: ")
            title = input("New Title (leave blank if unchanged): ")
            author = input("New Author (leave blank if unchanged): ")
            genre = input("New Genre (leave blank if unchanged): ")
            updates = {key: value for key, value in [("title", title), ("author", author), ("genre", genre)] if value}
            library.update_book(book_id, **updates)
        elif choice == '3':
            book_id = input("Book ID: ")
            library.remove_book(book_id)
        elif choice == '4':
            member_id = input("Member ID: ")
            name = input("Name: ")
            library.register_member(member_id, name)
        elif choice=='5':
            member_id = input("Member ID: ")
            name = input("New name : ")
            updates = {key: value for key, value in [("name", name)] if value}
            library.update_member(member_id, **updates) 
        elif choice =='6':
            member_id=input("Member ID:")
            library.remove_member(member_id)       
        elif choice == '7':
            book_id = input("Book ID: ")
            member_id = input("Member ID: ")
            library.borrow_book(book_id, member_id)
        elif choice == '8':
            book_id = input("Book ID: ")
            member_id = input("Member ID: ")
            library.return_book(book_id, member_id)
        elif choice == '9':
            library.list_borrowed_books()
        elif choice =='10':
            library.list_overdue_books()
        elif choice == '11':
            library.most_popular_books()
        elif choice == '12':
            search_term = input("Enter the book title you want to search for: ")
            results = library.search_books(search_term)
            if results:
                print("Book found:")
                for book in results:
                    print(book)
            else:
                print("No books found matching this search title!")
        elif choice == '13':
            search_term = input("Enter the member name you want to search for: ")
            results = library.search_members(search_term)
            if results:
                print("Member found:")
                for member in results:
                    print(member)
            else:
                print("No members found matching this  search name!")
        elif choice == '0':    
            break

if __name__ == "__main__":
    menu()


Library Management System
--------------------------
1.  Add book
2.  Update book
3.  Remove book
4.  Register member
5.  Update member
6.  Remove member
7.  Borrow book
8.  Return book
9.  List borrowed books
10. List overdue books
11. Most popular books
12. Search for book
13. Search for member
0.  Exit
--------------------------


Enter choice:  13
Enter the member name you want to search for:  fatma


Member found:
Member(ID: 5019, Name: 'fatma', Borrowed Books: [{'book_id': '20045', 'due_date': '2024-07-18'}])

Library Management System
--------------------------
1.  Add book
2.  Update book
3.  Remove book
4.  Register member
5.  Update member
6.  Remove member
7.  Borrow book
8.  Return book
9.  List borrowed books
10. List overdue books
11. Most popular books
12. Search for book
13. Search for member
0.  Exit
--------------------------
